In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
#!pip install scikit-learn==1.5.0
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import joblib

In [ ]:
#!pip list

In [ ]:
df = pd.read_csv('merged.csv')  # Ersetzen Sie 'data.csv' durch den Pfad Ihrer Datei

# Features und Labels extrahieren
features = df[['x', 'z', 'yaw', 'pitch', 'roll']].values
labels = df['action'].values

# Labels encodieren
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Feature-Skalierung
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(scaled_features, encoded_labels, test_size=0.2, random_state=42)

# Daten in Tensoren umwandeln
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoader erstellen
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [150]:
from torch.utils.data import Dataset
from itertools import chain

class IMU_Dataset(Dataset):
  def __init__(self, files, seq_size, feature_cols, label_col):
    self.files = files;
    self.df_concat = pd.concat([pd.read_csv(file) for file in files])
    self.seq_size = seq_size
    self.feature_cols = feature_cols
    self.label_col = label_col
    self.num_of_actions = 7
    self.next_possible_features = {
        "Neutral"        : [1, 1, 0, 1, 0, 1, 0],
        "Crouch_Forward" : [0, 1, 1, 0, 0, 0, 0],
        "Crouch_Backward": [1, 0, 1, 0, 0, 0, 0],
        "Jump_Forward"   : [0, 0, 0, 1, 1, 0, 0],
        "Jump_Backward"  : [1, 0, 0, 0, 1, 0, 0],
        "Shoot_Forward"  : [0, 0, 0, 0, 0, 1, 1],
        "Shoot_Backward" : [1, 0, 0, 0, 0, 0, 1]
      }
    self.action_map = {
        "Neutral"        : 0,
        "Crouch_Forward" : 1,
        "Crouch_Backward": 2,
        "Jump_Forward"   : 3,
        "Jump_Backward"  : 4,
        "Shoot_Forward"  : 5,
        "Shoot_Backward" : 6
      }

  def __len__(self):
    l = max(self.df_concat.shape[0] - self.seq_size, 0)
    return l

  def vote_majority_label(self, subset):
    key_count = {}
    actions = subset[self.label_col]
    #print(actions)
    for action in actions:
      #print(action)
      if action not in key_count:
        key_count[action] = 1
      else:
        key_count[action] += 1

    max_key = "Neutral"
    max_count = 0
    #print(key_count)
    for key in key_count.keys():
      if key_count[key] > max_count:
        max_key = key
        max_count = key_count[key]
    #print("max_key", max_key)
    return max_key

  def action_to_index(self, action):
    return self.action_map[action]

  def feature_function(self, subset, action):
    next_possible_feature = self.next_possible_features[action]
    #print(next_possible_feature)
    merged_cols = [subset[col].tolist() for col in subset.columns.values.tolist()]
    return list(chain.from_iterable(merged_cols)) + next_possible_feature

  def __getitem__(self, index): # index is start of seq_size
    if(index >= len(self)):
      raise Exception("index not in bounds")

    subset = self.df_concat.iloc[index : index+self.seq_size]
    subset_data = subset[self.feature_cols]
    #print(subset, "\n", subset_data)

    action = self.vote_majority_label(subset)
    #print(action)

    ff = self.feature_function(subset_data, action)
    #print(ff)
    return torch.tensor(ff), self.action_to_index(action)

  def getShapes(self):
    #print(self.seq_size, len(self.feature_cols), len(self.action_map.keys()))
    return [ self.seq_size * len(self.feature_cols) + len(self.action_map.keys()), self.num_of_actions]

In [201]:
class IMU_NNClassifier(nn.Module):
  def __init__(self, input_size, output_size, hidden_sizes,):
    super(IMU_NNClassifier, self).__init__()

    self.layers = nn.ModuleList()
    self.dropout = nn.Dropout(0.25)
    activation = nn.ReLU

    # Input layer
    self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
    self.layers.append(activation())

    # Hidden layers
    for i in range(1, len(hidden_sizes)):
        self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
        self.layers.append(activation())

    # Output layer
    self.layers.append(nn.Linear(hidden_sizes[-1], output_size))
    #self.layers.append(nn.Softmax(dim=1))

    # Combine all layers
    #self.network = nn.Sequential(*layers)

  def forward(self, x):
    for layer in self.layers:
      #print(x)
      x = layer(x)
    return x# torch.argmax(x, 1)


In [205]:
def train(model, criterion, optimizer, dataloader, epochs, clip_value=1.0):
  for epoch in range(epochs):
    total_loss = 0
    for i, (inputs, targets) in enumerate(dataloader):
      optimizer.zero_grad()
      outputs = model.forward(inputs)
      #print(outputs)
      #print(targets)

      loss = criterion(outputs, targets)

      loss.backward()
      optimizer.step()

      total_loss += loss.item()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

In [231]:
from itertools import repeat

imu_dataset = IMU_Dataset(
  ["/content/non_neutral_3xCrouch, Shoot, Crouch, Jump_labeled.csv"],
  7,
  ["x", "z", "yaw", "pitch", "roll"],
  "action"
)

[input_size, output_size]  = imu_dataset.getShapes()
hidden_sizes = list(repeat(input_size * 4, 3))
num_epochs = 100
learning_rate = 0.003
print(input_size, hidden_sizes, output_size)

imu_action_model = IMU_NNClassifier(input_size, output_size, hidden_sizes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(imu_action_model.parameters(), lr=learning_rate)

42 [168, 168, 168] 7


In [232]:
imu_dataloader = DataLoader(imu_dataset, batch_size=10, shuffle=False)

train(
    imu_action_model,
    criterion,
    optimizer,
    imu_dataloader,
    num_epochs
    )

Epoch [1/100], Loss: 110.3537
Epoch [2/100], Loss: 7.8452
Epoch [3/100], Loss: 23.9975
Epoch [4/100], Loss: 0.8903
Epoch [5/100], Loss: 0.0000
Epoch [6/100], Loss: 0.0000
Epoch [7/100], Loss: 0.0000
Epoch [8/100], Loss: 0.0000
Epoch [9/100], Loss: 29.7487
Epoch [10/100], Loss: 0.0002
Epoch [11/100], Loss: 1.7809
Epoch [12/100], Loss: 0.0000
Epoch [13/100], Loss: 12.6099
Epoch [14/100], Loss: 0.0003
Epoch [15/100], Loss: 10.3233
Epoch [16/100], Loss: 0.0000
Epoch [17/100], Loss: 0.0000
Epoch [18/100], Loss: 0.0000
Epoch [19/100], Loss: 0.0000
Epoch [20/100], Loss: 0.0000
Epoch [21/100], Loss: 0.0000
Epoch [22/100], Loss: 0.0000
Epoch [23/100], Loss: 0.0000
Epoch [24/100], Loss: 0.0000
Epoch [25/100], Loss: 0.0000
Epoch [26/100], Loss: 0.0000
Epoch [27/100], Loss: 0.0000
Epoch [28/100], Loss: 0.0000
Epoch [29/100], Loss: 0.0000
Epoch [30/100], Loss: 0.0000
Epoch [31/100], Loss: 0.0000
Epoch [32/100], Loss: 0.0000
Epoch [33/100], Loss: 0.0000
Epoch [34/100], Loss: 0.0000
Epoch [35/100], L

In [233]:
def evaluate_imu_model(model, criterion, dataloader):
  """
  model.eval()
  total_loss = 0.0
  with torch.no_grad():
      for inputs, targets in dataloader:
          outputs = model(inputs)
          loss = criterion(outputs, targets)
          total_loss += loss.item()

  avg_loss = total_loss / len(dataloader)
  print(f'Evaluation Loss: {avg_loss}')
  return avg_loss
  """
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0
    for X_batch, y_batch in dataloader:
        outputs = model.forward(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')

In [234]:
imu_test_dataset = IMU_Dataset(
  ["/content/3xCrouch, Shoot, Crouch, Jump_labeled.csv"],
  7,
  ["x", "z", "yaw", "pitch", "roll"],
  "action"
)
imu_test_dataloader = DataLoader(imu_test_dataset, batch_size=10, shuffle=False)
evaluate_imu_model(imu_action_model, criterion, imu_test_dataloader)


imu_test_dataset = IMU_Dataset(
  ["/content/non_neutral_3xCrouch, Shoot, Crouch, Jump_labeled.csv"],
  7,
  ["x", "z", "yaw", "pitch", "roll"],
  "action"
)

imu_test_dataloader = DataLoader(imu_test_dataset, batch_size=10, shuffle=False)
evaluate_imu_model(imu_action_model, criterion, imu_test_dataloader)

Accuracy: 7.34%
Accuracy: 98.54%


In [ ]:
# LSTM-Modell definieren
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
          h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
          c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [107]:
# Hyperparameter
input_size = 5  # Anzahl der Features
hidden_size = 128
output_size = len(label_encoder.classes_)
num_layers = 2
num_epochs = 10000
learning_rate = 0.001

# Modell, Verlustfunktion und Optimierer instanziieren
model = LSTMClassifier(input_size, hidden_size, output_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'label_encoder' is not defined

In [104]:
# Training des Modells
model.train()
for epoch in range(num_epochs):
  total_loss = 0
  for X_batch, y_batch in train_loader:
    X_batch = X_batch.unsqueeze(1)  # Batch-Dimension hinzufügen für LSTM
    outputs = model(X_batch)
    loss = criterion(outputs, y_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss}')

NameError: name 'model' is not defined

In [ ]:
# Save the trained model
import pickle
torch.save(model.state_dict(), 'gesture_model.pth')
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

with open('label_encoder.pkl','wb') as f:
    pickle.dump(label_encoder, f)

In [215]:
# Modell bewerten
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.unsqueeze(1)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')

NameError: name 'model' is not defined

In [ ]:
# Testen mit neuen Daten
def predict_new_data(new_data, model, scaler, label_encoder):
    # Neue Daten skalieren
    new_data_scaled = scaler.transform(new_data)

    # In Tensor umwandeln
    new_data_tensor = torch.tensor(new_data_scaled, dtype=torch.float32).unsqueeze(1)  # Batch-Dimension hinzufügen

    # Modell in den Evaluierungsmodus setzen
    model.eval()

    with torch.no_grad():
        outputs = model(new_data_tensor)
        _, predicted = torch.max(outputs.data, 1)

    # Vorhersagen decodieren
    predicted_labels = label_encoder.inverse_transform(predicted.cpu().numpy())

    return predicted_labels

In [ ]:
# Laden und Verfeinern des Modells
# Modell neu instanziieren und Zustand laden
model = LSTMClassifier(input_size, hidden_size, output_size, num_layers)
model.load_state_dict(torch.load('gesture_model.pth'))

# Optional: Modell weiter trainieren (feinabstimmen)
model.train()
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.unsqueeze(1)  # Batch-Dimension hinzufügen für LSTM
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Fine-tuning Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Speichern Sie das verfeinerte Modell
torch.save(model.state_dict(), 'refined_gesture_model.pth')

Fine-tuning Epoch [1/500], Loss: 0.0000
Fine-tuning Epoch [2/500], Loss: 0.0000
Fine-tuning Epoch [3/500], Loss: 0.0000
Fine-tuning Epoch [4/500], Loss: 0.0000
Fine-tuning Epoch [5/500], Loss: 0.0000
Fine-tuning Epoch [6/500], Loss: 0.0000
Fine-tuning Epoch [7/500], Loss: 0.0000
Fine-tuning Epoch [8/500], Loss: 0.0000
Fine-tuning Epoch [9/500], Loss: 0.0000
Fine-tuning Epoch [10/500], Loss: 0.0000
Fine-tuning Epoch [11/500], Loss: 0.0000
Fine-tuning Epoch [12/500], Loss: 0.0000
Fine-tuning Epoch [13/500], Loss: 0.0000
Fine-tuning Epoch [14/500], Loss: 0.0001
Fine-tuning Epoch [15/500], Loss: 0.0000
Fine-tuning Epoch [16/500], Loss: 0.0000
Fine-tuning Epoch [17/500], Loss: 0.0000
Fine-tuning Epoch [18/500], Loss: 0.0000
Fine-tuning Epoch [19/500], Loss: 0.0000
Fine-tuning Epoch [20/500], Loss: 0.0000
Fine-tuning Epoch [21/500], Loss: 0.0000
Fine-tuning Epoch [22/500], Loss: 0.0000
Fine-tuning Epoch [23/500], Loss: 0.0000
Fine-tuning Epoch [24/500], Loss: 0.0000
Fine-tuning Epoch [25/500